In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torch.nn.functional as F
from torchvision.datasets import ImageFolder

In [33]:
# Combine the components into the complete model
class LandCoverModel(nn.Module):
    def __init__(self, backbone, weakly_supervised_module):
        super(LandCoverModel, self).__init__()
        self.backbone = backbone
        self.weakly_supervised_module = weakly_supervised_module

    def forward(self, x):
        features = self.backbone(x)
        logits = self.weakly_supervised_module(features)
        return logits

In [24]:
# Data loading and preprocessing
data_dir = 'Feb_17_2024_workspace/Tile256/'
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

In [25]:
# Use torchvision's ImageFolder to load the dataset
your_dataset = ImageFolder(root=data_dir, transform=transform)

In [26]:
# Split the dataset into training and validation sets (adjust as needed)
train_size = int(0.8 * len(your_dataset))
val_size = len(your_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(your_dataset, [train_size, val_size])

In [27]:
# Create data loaders
batch_size = 32
#train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

import torch

def custom_collate(batch):
    # Resize images to a fixed size
    batch = [(torch.nn.functional.interpolate(image.unsqueeze(0), size=(256, 256), mode='bilinear', align_corners=False)).squeeze(0) for image in batch]
    return torch.stack(batch, dim=0)

# Update your DataLoader to use the custom collate function
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate)


import torch

def custom_collate(batch):
    # Resize images to a fixed size
    batch = [(torch.nn.functional.interpolate(image[0].unsqueeze(0), size=(256, 256), mode='bilinear', align_corners=False)).squeeze(0) for image in batch]
    return torch.stack(batch, dim=0)

# Update your DataLoader to use the custom collate function
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate)


In [28]:
import torch

def custom_collate(batch):
    # Resize images to a fixed size
    resized_images = [(torch.nn.functional.interpolate(image.unsqueeze(0), size=(256, 256), mode='bilinear', align_corners=False)).squeeze(0) for image, _ in batch]
    labels = [label for _, label in batch]
    return torch.stack(resized_images, dim=0), torch.tensor(labels)

# Update your DataLoader to use the custom collate function
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate)


In [29]:
# Instantiate the components
backbone = ResolutionPreservingBackbone()
weakly_supervised_module = WeaklySupervisedModule(num_classes=len(your_dataset.classes))
contrastive_loss_fn = ContrastiveLoss()
model = LandCoverModel(backbone, weakly_supervised_module)

In [30]:
# Set up optimizer and training parameters
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10

In [ ]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        logits = model(images)
        
        # Example weakly supervised loss (adjust as needed)
        weak_loss = nn.CrossEntropyLoss()(logits, labels)
        
        # Example self-supervised contrastive loss
        contrastive_loss = contrastive_loss_fn(logits)
        
        # Combine the losses based on your specific strategy
        total_loss = weak_loss + contrastive_loss
        
        total_loss.backward()
        optimizer.step()

    # Validation (optional)
    model.eval()
    with torch.no_grad():
        for val_images, val_labels in val_loader:
            val_logits = model(val_images)
            # Validation logic, if needed

    # Print or log training statistics
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss.item()}')


In [ ]:
""""
TRAINING MODEL 2
"""
# Training loop...................model2.........2......Training....

for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        logits = model(images)
        
        # Example weakly supervised loss (adjust as needed)
        weak_loss = nn.CrossEntropyLoss()(logits, labels)

        # Example self-supervised contrastive loss
        contrastive_loss = contrastive_loss_fn(logits)

        # Combine the losses based on your specific strategy
        total_loss = weak_loss + contrastive_loss

        total_loss.backward()
        optimizer.step()

    # Validation (optional)
    model.eval()
    with torch.no_grad():
        for val_images, val_labels in val_loader:
            val_logits = model(val_images)
            # Validation logic, if needed

    # Print or log training statistics
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss.item()}')

# Save or use the trained model for land-cover mapping
torch.save(model.state_dict(), 'land_cover_model.pth')

In [ ]:
#..................add()..................METRICES..................
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, accuracy_score, cohen_kappa_score

""""
# Define the LandCoverModel, ResolutionPreservingBackbone, WeaklySupervisedModule, 
# and ContrastiveLoss as in the previous example
"""

# Function to calculate Intersection over Union (IoU)
def calculate_iou(y_true, y_pred):
    intersection = np.logical_and(y_true, y_pred)
    union = np.logical_or(y_true, y_pred)
    iou = np.sum(intersection) / np.sum(union)
    return iou

# Function to evaluate the model on the validation set
def evaluate_model(model, val_loader, device):
    model.eval()
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for val_images, val_labels in val_loader:
            val_images, val_labels = val_images.to(device), val_labels.to(device)
            logits = model(val_images)
            predictions = torch.argmax(logits, dim=1)
            
            all_labels.extend(val_labels.cpu().numpy())
            all_predictions.extend(predictions.cpu().numpy())

    return np.array(all_labels), np.array(all_predictions)

# Function to plot confusion matrix
def plot_confusion_matrix(y_true, y_pred, classes, save_path):
    cm = confusion_matrix(y_true, y_pred, normalize='true')
    plt.figure(figsize=(10, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.savefig(save_path, dpi=1000)
    plt.close()

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


""""
THE COMPLETE TRAINING AND PRODUCING ALL METRICES TO EVALUATE MODEL PERFORMANCE
"""
# Training loop with evaluation and plotting
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        logits = model(images)
        loss = your_loss_function(logits, labels)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_labels, val_predictions = evaluate_model(model, val_loader, device)

    # Calculate metrics
    iou = calculate_iou(val_labels, val_predictions)
    f1 = f1_score(val_labels, val_predictions, average='weighted')
    accuracy = accuracy_score(val_labels, val_predictions)
    kappa = cohen_kappa_score(val_labels, val_predictions)

    # Print metrics
    print(f'Epoch [{epoch+1}/{num_epochs}], IoU: {iou:.4f}, F1-Score: {f1:.4f}, Accuracy: {accuracy:.4f}, Kappa: {kappa:.4f}')

    # Plot and save confusion matrix
    plot_confusion_matrix(val_labels, val_predictions, your_class_labels, f'confusion_matrix_epoch_{epoch+1}.png')

# Save or use the trained model for land-cover mapping
torch.save(model.state_dict(), 'land_cover_model.pth')